In [2]:
import requests

url = "https://api.upbit.com/v1/market/all"

querystring = {"isDetails":"false"}

response = requests.request("GET", url, params=querystring)

print(response.json())

{'market': 'BTC-ADX', 'korean_name': '애드엑스', 'english_name': 'AdEx'}, {'market': 'BTC-SYS', 'korean_name': '시스코인', 'english_name': 'SysCoin'}, {'market': 'BTC-ANT', 'korean_name': '아라곤', 'english_name': 'Aragon'}, {'market': 'BTC-STORJ', 'korean_name': '스토리지', 'english_name': 'Storj'}, {'market': 'BTC-GRS', 'korean_name': '그로스톨코인', 'english_name': 'Groestlcoin'}, {'market': 'BTC-REP', 'korean_name': '어거', 'english_name': 'Augur'}, {'market': 'BTC-RLC', 'korean_name': '아이젝', 'english_name': 'iEx.ec'}, {'market': 'BTC-EMC2', 'korean_name': '아인스타이늄', 'english_name': 'Einsteinium'}, {'market': 'USDT-BTC', 'korean_name': '비트코인', 'english_name': 'Bitcoin'}, {'market': 'USDT-ETH', 'korean_name': '이더리움', 'english_name': 'Ethereum'}, {'market': 'USDT-LTC', 'korean_name': '라이트코인', 'english_name': 'Litecoin'}, {'market': 'USDT-XRP', 'korean_name': '리플', 'english_name': 'Ripple'}, {'market': 'USDT-ETC', 'korean_name': '이더리움클래식', 'english_name': 'Ethereum Classic'}, {'market': 'KRW-NEO', 'korean_na

In [136]:
# 분 켄들 조회
import requests
import json
import datetime
import math
import numpy as np

url = "https://api.upbit.com/v1/candles/minutes/3"

get_count = 200
#querystring = {"market":"KRW-XRP","to":"2021-02-05T00:00:00Z", "count": str(get_count)}
querystring = {"market":"KRW-XRP","count":get_count}

response = requests.request("GET", url, params=querystring)
#print(response.text)
result = json.loads(response.text)

scores = []
for i in range(0, get_count):
    item = result[i]
    cur_time = datetime.datetime.fromtimestamp(item["timestamp"]/1000)
    gap = math.fabs(item["opening_price"] - item["trade_price"])
    #gap = item["high_price"] - item["low_price"]
    prev_volume = 0
    #if i < (get_count-1):
    #    prev_volume = result[i+1]["candle_acc_trade_volume"]
    gap_volume = math.fabs(item["candle_acc_trade_volume"] - prev_volume)
    diff_score = (gap if gap > 0.0 else 0.1) * (gap_volume * 0.001)
    scores.append(diff_score)

    #print(i, item["candle_date_time_kst"], cur_time.strftime("%m/%d %H:%M:%S"), gap, item["trade_price"], item["candle_acc_trade_volume"], 
    #    int(gap_volume), int(diff_score))

scores = np.array(scores)
std = np.std(scores)
avg =  np.average(scores)

print('avg score :', avg, std)

std_diff = scores - std
idxs = np.where(std_diff > std)
for i in idxs[0]:
    item = result[i]
    cur_time = datetime.datetime.fromtimestamp(item["timestamp"]/1000)
    gap = math.fabs(item["opening_price"] - item["trade_price"])
    #gap = item["high_price"] - item["low_price"]

    print(i, item["candle_date_time_kst"], cur_time.strftime("%m/%d %H:%M:%S"), gap, item["trade_price"], gap / item["trade_price"] * 100, item["candle_acc_trade_volume"])

#print('sort:', np.sort(std_diff))


avg score : 940.5848926447807 2247.685904426096
2 2021-02-07T00:12:00 02/07 00:15:00 2.0 473.0 0.42283298097251587 2912213.39874642
9 2021-02-06T23:51:00 02/06 23:54:00 4.0 473.0 0.8456659619450317 5278907.09351184
62 2021-02-06T21:12:00 02/06 21:15:07 4.0 463.0 0.8639308855291578 3078385.73800484
72 2021-02-06T20:42:00 02/06 20:44:59 2.0 469.0 0.42643923240938164 2472939.56479021
73 2021-02-06T20:39:00 02/06 20:41:59 4.0 471.0 0.8492569002123143 3497321.60530839
82 2021-02-06T20:12:00 02/06 20:14:59 2.0 467.0 0.4282655246252677 2264452.91235247
191 2021-02-06T14:45:00 02/06 14:47:58 3.0 470.0 0.6382978723404255 3666489.17472251


In [125]:
idxs = np.where(std_diff > 100000)
for i in idxs[0]:
    item = result[i]
    cur_time = datetime.datetime.fromtimestamp(item["timestamp"]/1000)
    gap = math.fabs(item["opening_price"] - item["trade_price"])
    #gap = item["high_price"] - item["low_price"]

    print(i, item["candle_date_time_kst"], cur_time.strftime("%m/%d %H:%M:%S"), gap, item["trade_price"], gap / item["trade_price"] * 100, item["candle_acc_trade_volume"])


36 2021-02-06T21:10:00 02/06 21:15:07 5.0 463.0 1.079913606911447 3801867.10145222
159 2021-02-06T10:55:00 02/06 11:00:00 5.0 462.0 1.0822510822510822 9017688.60796993
160 2021-02-06T10:50:00 02/06 10:55:00 6.0 457.0 1.312910284463895 17208749.90240401
161 2021-02-06T10:45:00 02/06 10:49:59 4.0 463.0 0.8639308855291578 8320409.19783528
164 2021-02-06T10:30:00 02/06 10:34:59 3.0 467.0 0.6423982869379015 7207806.84094944


In [12]:
# 현 시세 조회
import requests
import json

url = "https://api.upbit.com/v1/ticker"
querystring = {"markets":"KRW-XRP"}
response = requests.request("GET", url, params=querystring)
result = json.loads(response.text)
print(json.dumps(result))
print(result[0]["trade_price"], result[0]["change"])

[{"market": "KRW-XRP", "trade_date": "20210205", "trade_time": "160959", "trade_date_kst": "20210206", "trade_time_kst": "010959", "trade_timestamp": 1612541399000, "opening_price": 481.0, "high_price": 508.0, "low_price": 454.0, "trade_price": 488.0, "prev_closing_price": 481.0, "change": "RISE", "change_price": 7.0, "change_rate": 0.0145530146, "signed_change_price": 7.0, "signed_change_rate": 0.0145530146, "trade_volume": 11.02459016, "acc_trade_price": 454588773309.55804, "acc_trade_price_24h": 850287008343.7313, "acc_trade_volume": 951089244.7500465, "acc_trade_volume_24h": 1788767953.1521726, "highest_52_week_price": 859.0, "highest_52_week_date": "2020-11-24", "lowest_52_week_price": 155.0, "lowest_52_week_date": "2020-03-13", "timestamp": 1612541399611}]
488.0 RISE


In [13]:
# telegram bot api test
import requests

url = "https://api.telegram.org/bot1640815585:AAHancwBwckiHtRFTc2g78akroVDYTtU1E4/getUpdates"
response = requests.request("GET", url)
updates = response.json()
print(updates["result"])


[{'update_id': 131941975, 'message': {'message_id': 11, 'from': {'id': 1693965788, 'is_bot': False, 'first_name': 'ksjin'}, 'chat': {'id': 1693965788, 'first_name': 'ksjin', 'type': 'private'}, 'date': 1612766081, 'text': '/start', 'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}}, {'update_id': 131941976, 'message': {'message_id': 13, 'from': {'id': 1693965788, 'is_bot': False, 'first_name': 'ksjin', 'language_code': 'ko'}, 'chat': {'id': 1693965788, 'first_name': 'ksjin', 'type': 'private'}, 'date': 1612779166, 'text': '/'}}, {'update_id': 131941977, 'message': {'message_id': 14, 'from': {'id': 1693965788, 'is_bot': False, 'first_name': 'ksjin', 'language_code': 'ko'}, 'chat': {'id': 1693965788, 'first_name': 'ksjin', 'type': 'private'}, 'date': 1612779205, 'text': '/start', 'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}}, {'update_id': 131941978, 'message': {'message_id': 15, 'from': {'id': 1693965788, 'is_bot': False, 'first_name': 'ksjin', 'langua

In [14]:
idlist = set()
for msg in updates["result"]:
    idlist.add(msg["message"]["chat"]["id"])
print(idlist)    

#print(updates["result"][0]["message"]["chat"])

{547008236, 1605321409, 1693965788, 678478220}


In [18]:
# 분 켄들 조회
import requests
import json
import datetime
import math
import numpy as np

# ripple
target_markget = "KRW-XRP"
total_minutes = 10

def GetMinutesData(market, count=20):
    url = "https://api.upbit.com/v1/candles/minutes/1"
    #querystring = {"market":"KRW-XRP","to":"2021-02-05T00:00:00Z", "count": str(get_count)}
    querystring = {"market":market,"count":count}
    response = requests.request("GET", url, params=querystring)
    if response.status_code == 200:
        return response.json()
    
    #print(response.text)

    return None

def GetCurrentPrice(market):
    url = "https://api.upbit.com/v1/ticker"
    querystring = {"markets":market}
    response = requests.request("GET", url, params=querystring)
    if response.status_code != 200:
        return None, None

    result = response.json()

    #print(json.dumps(result))
    #print(result[0]["trade_price"], result[0]["change"])
    cur_time = datetime.datetime.fromtimestamp(result[0]["timestamp"]/1000)

    return cur_time, result[0]["trade_price"]


mins_datas = GetMinutesData(target_markget, total_minutes)
cur_time, cur_price = GetCurrentPrice(target_markget)

#print(mins_datas)
#print(cur_price)

alert_percent = 0.01
for item in mins_datas:
    item_time = datetime.datetime.fromtimestamp(item["timestamp"]/1000)
    diff_time = cur_time - item_time
    if diff_time.total_seconds() < 59:
        continue

    trade_price = item["trade_price"]
    diff_price = cur_price - trade_price

    percent = math.fabs(diff_price) / trade_price

    if percent > alert_percent:
        # alert
        print(f'{diff_time} {trade_price}/{diff_price} {percent * 100:0.2f}')
        break

    #print(i, item["candle_date_time_kst"], cur_time.strftime("%m/%d %H:%M:%S"), gap, item["trade_price"], gap / item["trade_price"] * 100, item["candle_acc_trade_volume"])

#print(price_volumes)

466.0


In [2]:
from telegram_api import SendMessage

SendMessage("API 설정 실수로 저한테만 메시지를 보내고 있었네요 ㅋㅋ")

{547008236, 1605321409, 1693965788, 678478220}
SendMessage is successed
{'ok': True, 'result': {'message_id': 31, 'from': {'id': 1640815585, 'is_bot': True, 'first_name': 'oioioi', 'username': 'oioioinoti_bot'}, 'chat': {'id': 547008236, 'first_name': 'Jeffrey', 'last_name': 'Im', 'type': 'private'}, 'date': 1612798520, 'text': '안녕~! 테스트임다'}}
SendMessage is successed
{'ok': True, 'result': {'message_id': 32, 'from': {'id': 1640815585, 'is_bot': True, 'first_name': 'oioioi', 'username': 'oioioinoti_bot'}, 'chat': {'id': 1605321409, 'first_name': 'Grek', 'type': 'private'}, 'date': 1612798521, 'text': '안녕~! 테스트임다'}}
SendMessage is successed
{'ok': True, 'result': {'message_id': 33, 'from': {'id': 1640815585, 'is_bot': True, 'first_name': 'oioioi', 'username': 'oioioinoti_bot'}, 'chat': {'id': 1693965788, 'first_name': 'ksjin', 'type': 'private'}, 'date': 1612798523, 'text': '안녕~! 테스트임다'}}
SendMessage is successed
{'ok': True, 'result': {'message_id': 34, 'from': {'id': 1640815585, 'is_bo